In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import math
import multiprocessing
from matplotlib import pyplot as plt

import keras
from keras import layers, models
from keras import Input
from keras.models import Model
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
color_label = {'빨강': 0, '주황': 1, '노랑': 2, '연두': 3, '초록': 4, 
               '청록': 5, '파랑': 6, '남색': 7, '보라': 8, '분홍': 9,
               '자주': 10, '갈색': 11, '회색': 12, '검정': 13, '하양': 14, '투명': 15}

In [7]:
params = {
    'img_size': (224, 224),
    'input_shape': (224, 224, 3),
    'batch_size': 24,
    'epochs': 100,
    'nb_workers': multiprocessing.cpu_count()
}

In [8]:
DATA_PATH = '../data/mask_front'
LABEL_PATH = '../label'
SAVE_PATH = '../save_model'

In [9]:
train_datagen=ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator=train_datagen.flow_from_directory(
    '../aug_data/aug_color2/',
    subset='training',
    class_mode='categorical',
    target_size=params['img_size'],
    batch_size=params['batch_size'])

valid_generator=train_datagen.flow_from_directory(
    '../aug_data/aug_color2/',
    subset='validation',
    class_mode='categorical',
    target_size=params['img_size'],
    batch_size=params['batch_size'])

Found 106798 images belonging to 16 classes.
Found 26694 images belonging to 16 classes.


In [10]:
# top-k
from functools import partial
top_2 = partial(keras.metrics.top_k_categorical_accuracy, k=2)
top_2.__name__ = 'top_2'

top_3 = partial(keras.metrics.top_k_categorical_accuracy, k=3)
top_3.__name__ = 'top_3'

top_5 = keras.metrics.top_k_categorical_accuracy
top_5.__name__ = 'top_5'

top_10 = partial(keras.metrics.top_k_categorical_accuracy, k=10)
top_10.__name__ = 'top_10'

In [11]:
######################## VGG16 #########################
from keras.applications import VGG16
cnn_model = VGG16(include_top=False, weights='imagenet', input_shape=params['input_shape'])
cnn_model.trainable = True
model = Sequential()
model.add(cnn_model)
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='softmax', kernel_initializer='he_normal'))
model.summary()

# ###
# dependencies = {
#     'top_2': top_2,
#     'top_3': top_3,
#     'top_5': top_5
# }

# model = keras.models.load_model(os.path.join(SAVE_PATH, 'VGG16_color_ep005_vloss-1.3428_vacc-0.6989.h5'), custom_objects=dependencies)
# ###
filepath = os.path.join(SAVE_PATH, 'VGG16_color_ep{epoch:03d}_vloss-{val_loss:.4f}_vacc-{val_acc:.4f}.h5')

checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc', verbose=1, save_best_only=False)
# earlystop = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')

model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc', top_2, top_3])
# model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=2e-6), metrics=['acc', top_2, top_3])

history = model.fit_generator(train_generator,
                              steps_per_epoch = train_generator.samples // params['batch_size'],
                              validation_data=valid_generator, 
                              validation_steps = valid_generator.samples // params['batch_size'],
                              epochs=params['epochs'],
                              callbacks=[checkpoint],
                              workers=params['nb_workers'])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              4195328   
_________________________________________________________________
dropout_2 (Dropout)  

Epoch 21/100
4449/4449 [==============================] - 1263s 284ms/step - loss: 15.1043 - acc: 0.0629 - top_2: 0.9999 - top_3: 1.0000 - val_loss: 15.1469 - val_acc: 0.0603 - val_top_2: 1.0000 - val_top_3: 1.0000

Epoch 00021: saving model to ../save_model\VGG16_color_ep021_vloss-15.1469_vacc-0.0603.h5
Epoch 22/100
4449/4449 [==============================] - 1263s 284ms/step - loss: 15.1000 - acc: 0.0632 - top_2: 1.0000 - top_3: 1.0000 - val_loss: 15.1378 - val_acc: 0.0608 - val_top_2: 1.0000 - val_top_3: 1.0000

Epoch 00022: saving model to ../save_model\VGG16_color_ep022_vloss-15.1378_vacc-0.0608.h5
Epoch 23/100
4449/4449 [==============================] - 1263s 284ms/step - loss: 15.1048 - acc: 0.0629 - top_2: 0.9999 - top_3: 1.0000 - val_loss: 15.1511 - val_acc: 0.0600 - val_top_2: 1.0000 - val_top_3: 1.0000

Epoch 00023: saving model to ../save_model\VGG16_color_ep023_vloss-15.1511_vacc-0.0600.h5
Epoch 24/100
4449/4449 [==============================] - 1263s 284ms/step - loss:

Epoch 48/100
4449/4449 [==============================] - 1264s 284ms/step - loss: 15.1204 - acc: 0.0619 - top_2: 1.0000 - top_3: 1.0000 - val_loss: 15.1445 - val_acc: 0.0604 - val_top_2: 1.0000 - val_top_3: 1.0000

Epoch 00048: saving model to ../save_model\VGG16_color_ep048_vloss-15.1445_vacc-0.0604.h5
Epoch 49/100
4449/4449 [==============================] - 1264s 284ms/step - loss: 15.1138 - acc: 0.0623 - top_2: 1.0000 - top_3: 1.0000 - val_loss: 15.1523 - val_acc: 0.0599 - val_top_2: 1.0000 - val_top_3: 1.0000

Epoch 00049: saving model to ../save_model\VGG16_color_ep049_vloss-15.1523_vacc-0.0599.h5
Epoch 50/100
4449/4449 [==============================] - 1264s 284ms/step - loss: 15.1289 - acc: 0.0614 - top_2: 1.0000 - top_3: 1.0000 - val_loss: 15.1529 - val_acc: 0.0599 - val_top_2: 1.0000 - val_top_3: 1.0000

Epoch 00050: saving model to ../save_model\VGG16_color_ep050_vloss-15.1529_vacc-0.0599.h5
Epoch 51/100
4449/4449 [==============================] - 1264s 284ms/step - loss:

Epoch 75/100
4449/4449 [==============================] - 1263s 284ms/step - loss: 15.1138 - acc: 0.0623 - top_2: 1.0000 - top_3: 1.0000 - val_loss: 15.1487 - val_acc: 0.0601 - val_top_2: 1.0000 - val_top_3: 1.0000

Epoch 00075: saving model to ../save_model\VGG16_color_ep075_vloss-15.1487_vacc-0.0601.h5
Epoch 76/100
4449/4449 [==============================] - 1263s 284ms/step - loss: 15.1241 - acc: 0.0617 - top_2: 1.0000 - top_3: 1.0000 - val_loss: 15.1560 - val_acc: 0.0597 - val_top_2: 1.0000 - val_top_3: 1.0000

Epoch 00076: saving model to ../save_model\VGG16_color_ep076_vloss-15.1560_vacc-0.0597.h5
Epoch 77/100
4449/4449 [==============================] - 1263s 284ms/step - loss: 15.1094 - acc: 0.0626 - top_2: 1.0000 - top_3: 1.0000 - val_loss: 15.1475 - val_acc: 0.0602 - val_top_2: 1.0000 - val_top_3: 1.0000

Epoch 00077: saving model to ../save_model\VGG16_color_ep077_vloss-15.1475_vacc-0.0602.h5
Epoch 78/100
4449/4449 [==============================] - 1263s 284ms/step - loss:

In [ ]:
def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0 :
        return (num_samples // batch_size) + 1
    else :
        return num_samples // batch_size

In [ ]:
dataset_test = pd.read_csv(os.path.join(LABEL_PATH,'shape_color_prediction.csv'))
dataset_test = dataset_test[['No']]

from keras.models import load_model
params.update({
    'nb_test_samples': len(dataset_test)
})

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=dataset_test,
    directory=DATA_PATH,
    x_col='No',
    y_col=None,
    target_size=params['img_size'],
    class_mode=None,
    batch_size=params['batch_size'],
    shuffle=False)

dependencies = {
    'top_2': top_2,
    'top_3': top_3,
    'top_5': top_5
}

model = keras.models.load_model(os.path.join(SAVE_PATH, 'VGG16_color_ep005_vloss-1.3428_vacc-0.6989.h5'), custom_objects=dependencies)



prediction = model.predict_generator(generator = test_generator,
                                     steps = get_steps(params['nb_test_samples'], params['batch_size']),
                                     verbose=1,
                                     workers=params['nb_workers'])

In [ ]:
len(prediction)

In [ ]:
predicted_class_indices=np.argmax(prediction, axis=1)

# Generator class dictionary mapping
labels = (train_generator.class_indices)
labels = dict((v, k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
len(predictions)
submission = pd.read_csv(os.path.join(LABEL_PATH, 'shape_color_prediction.csv'))
submission['aug_color_top1'] = predictions
submission.to_csv(os.path.join(LABEL_PATH, 'shape_color_prediction.csv'), index=False)

In [ ]:
def top_k_label(K):
    top_list = []
    predicted_class_indices=np.argmax(prediction, axis=1)
    labels = (train_generator.class_indices)
    labels = dict((v,k) for k,v in labels.items())
    class_probs = prediction
    for i, l in enumerate(predictions): # idx, label
        class_prob = class_probs[i]
        top_values = (-class_prob).argsort()[:K].tolist() # k 개까지 높은 확률 인덱스 저장
        top_list.append(top_values)
        
    top_arr = np.zeros((len(top_list), K))
    
    for i in range(len(top_list)): # key - values 값 바꾸기
        for j in range(K):
            tmp = top_list[i][j]
            top_arr[i][j] = labels[tmp]
    
    label_list = top_arr.tolist()
    return label_list


# top_k_label(3)

In [ ]:
# # top-k csv파일에 저장하기

# df = pd.read_csv(os.path.join(LABEL_PATH,'shape_color_prediction.csv'))
# df['aug_color_top3'] = top_k_label(3)
# df.to_csv(os.path.join(LABEL_PATH, 'shape_color_prediction.csv'), index=False)
# df.head()

In [ ]:
df = pd.read_csv(os.path.join(LABEL_PATH,'shape_color_prediction.csv'))
count = 0
for i in range(len(df)):
    if df['color_front'][i] == df['aug_color_top1'][i]:
        count += 1
print(count/len(df))